In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from sklearn.model_selection import train_test_split

import re
import string

In [113]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
df_path = '../data/clean_recipes.csv'
df = pd.read_csv(df_path, sep=';')

In [2]:
kgl1_df = pd.read_json('../data/kgl_ingredient_train.json')
kgl2_df = pd.read_json('../data/kgl_ingredient_test.json')

NameError: name 'pd' is not defined

In [7]:
print(df.shape)
df.head()

(12351, 10)


,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003
3,Apple Raisin Bread Recipe,39,https://images.media-allrecipes.com/userphotos...,Helen Hanson,15 m,1 h,1 h 15 m,"flour,baking powder,baking soda,salt,cinnamon,...",Preheat oven to 350 degrees F (175 degrees C)....,7006
4,Buttermilk Oatmeal Bread Recipe,41,https://images.media-allrecipes.com/userphotos...,Helen Hanson,10 m,1 h,1 h 40 m,"oat,buttermilk,vegetable oil,egg,brown sugar,f...",Mix oats with buttermilk. Let stand for 1/2 h...,7007


In [12]:
ingr_series = df['Ingredients']
kgl1_series = kgl1_df['ingredients']
kgl2_series = kgl2_df['ingredients']

In [16]:
type(ingr_series[0]), type(kgl1_series[0]), type(kgl2_series[0])

(str, list, list)

In [17]:
ingr_series = ingr_series.apply(lambda x: x.split(','))

In [19]:
type(ingr_series[0]), type(kgl1_series[0]), type(kgl2_series[0])

(list, list, list)

In [20]:
print(len(ingr_series), len(kgl1_series), len(kgl2_series))

12351 39774 9944


In [21]:
ingr_list = ingr_series.append([kgl1_series, kgl2_series], ignore_index=True)
print(len(ingr_list))
ingr_list.head()

62069


0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
dtype: object

In [52]:
ingredients = set(x for l in ingr_list for x in l)
ingredient_list = list(ingredients)

ingredient_list

lets',
 'Dry Mix Ingredients',
 'rum syrup',
 'uncooked lasagna',
 'sugar snap pea',
 'pickled vegetables',
 'apricot jelly',
 'egg beaters',
 'pomelo',
 'grate lime peel',
 'agar',
 'king salmon',
 'instant rice',
 'anise powder',
 'melon liqueur',
 '1 (10 ounce) can bamboo shoots',
 'grilled chicken',
 'creamed coconut',
 'miso sesame grilling sauce',
 'coleslaw',
 'green bell pepper',
 'grapefruit',
 'crumbled cornbread',
 'icing mix',
 'splenda granular',
 'sweet mustard',
 'canned chicken broth',
 'herb',
 'bermuda onion',
 'all purpose unbleached flour',
 'cherries',
 'queso ranchero',
 'greek yogurt',
 'yeast',
 'soppressata',
 'japanese eggplants',
 'chile verde',
 "za'atar",
 'Barilla Linguine',
 'blackberries',
 'onion salt',
 'pork sausages',
 'medium tomatoes',
 'McCormick Ground White Pepper',
 'chicken bouillon',
 'hungarian hot paprika',
 'soybean paste',
 'crosswise',
 'diced red onions',
 'almond liqueur',
 'unsweetened vanilla almond milk',
 'rose petals',
 'bosc pear

In [178]:
ingredient_list

lets',
 'Dry Mix Ingredients',
 'rum syrup',
 'uncooked lasagna',
 'sugar snap pea',
 'pickled vegetables',
 'apricot jelly',
 'egg beaters',
 'pomelo',
 'grate lime peel',
 'agar',
 'king salmon',
 'instant rice',
 'anise powder',
 'melon liqueur',
 '1 (10 ounce) can bamboo shoots',
 'grilled chicken',
 'creamed coconut',
 'miso sesame grilling sauce',
 'coleslaw',
 'green bell pepper',
 'grapefruit',
 'crumbled cornbread',
 'icing mix',
 'splenda granular',
 'sweet mustard',
 'canned chicken broth',
 'herb',
 'bermuda onion',
 'all purpose unbleached flour',
 'cherries',
 'queso ranchero',
 'greek yogurt',
 'yeast',
 'soppressata',
 'japanese eggplants',
 'chile verde',
 "za'atar",
 'Barilla Linguine',
 'blackberries',
 'onion salt',
 'pork sausages',
 'medium tomatoes',
 'McCormick Ground White Pepper',
 'chicken bouillon',
 'hungarian hot paprika',
 'soybean paste',
 'crosswise',
 'diced red onions',
 'almond liqueur',
 'unsweetened vanilla almond milk',
 'rose petals',
 'bosc pear

In [115]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [125]:
def ingredient_count(dfcolumn):
    '''Parses a column of word lists and returns a dictionary of words/wordcounts'''

    # If observation is one long string of ingredients, split by comma
    if type(dfcolumn[0]) == str:
        dfcolumn = dfcolumn.apply(lambda x: x.split(','))

    # Keep only letters (and ' for potential possessives)
    char_table = str.maketrans('', '', string.punctuation)
    dfcolumn = [[word.translate(char_table) for word in word_list] for word_list in dfcolumn if word_list]

    # convert all to lower_case
    for row in dfcolumn:
        row = [i.lower() for i in row]

    # Drop measurement and conjunction words
    dropwords = ['ounce', 'ounces', 'oz', 'oz.', 'oz.', 'lb', 'lb.', ' pound', 'package', 'packages', 'inches',
                 'ground', 'and', 'for', 'teaspoon', 'tablespoon', 'tbsp', 'tsp']
    dfcolumn = [[word.lower().strip(' .:"()1234567890%/\\\'') for word in word_list if word.lower().strip(' .:"()1234567890%/\\\'') not in dropwords] for word_list in dfcolumn if word_list]

    # Create dictionary with word count
    d = dict()

    for word_list in dfcolumn:
        for word in word_list:
            if word in d:
                d[word] = d[word] + 1
            else:
                d[word] = 1

    return d

In [127]:
ingr_count = ingredient_count(ingrDF['item'])

In [118]:
ingrDF = pd.DataFrame(ingr_count.items(), columns=['item', 'total_count'])
ingrDF = ingrDF.sort_values('total_count', ascending=False).reset_index(drop=True)
ingrDF.head(15)

,item,total_count
0,salt,27050
1,water,11899
2,garlic,11287
3,butter,10534
4,onions,10008
5,olive oil,9889
6,sugar,9319
7,garlic cloves,7772
8,pepper,7080
9,vegetable oil,6874


In [86]:
top_ingr = ingrDF[ingrDF.total_count > 0]
len(top_ingr)

7654

In [1]:
ingrDF[ingrDF['item'] == 'salt']

NameError: name 'ingrDF' is not defined

In [120]:
derptest = list(set(ingrDF['item']))
print(len(derptest))
derptest[500:550]

7654


['pumpkin pie filling',
 'pistou',
 'cornonthecob',
 'toulouse sausage',
 'date',
 'feta cheese',
 'napa cabbage leaves',
 'umeboshi plum vinegar',
 'seafood glaze',
 'cane vinegar',
 'soy glaze',
 'soft taco size flour tortillas',
 'skinless boneless chicken legs',
 'crumpet',
 'mixed spice',
 'frozen hash browns',
 'margarine spread',
 'toasted walnuts',
 'split peas',
 'frosting',
 'confit',
 'bitter gourd',
 'nonfat dry milk',
 'knorr chicken flavor bouillon',
 'white bread flour',
 'refrigerated bread dough',
 'chopped cilantro fresh',
 'frozen chicken wings',
 'frozen lemonade concentrate',
 'marsala wine',
 'peanut brittle',
 'cooking sherry',
 'lowfat cheddar',
 'pullman loaf',
 'calf liver',
 'dried beans',
 'accent seasoning',
 'fresh ginger root',
 'stolichnaya',
 'boneless chicken breast',
 'aged cheddar cheese',
 'whitefish',
 'guinea hens',
 'gluten free lasagna noodle',
 'pink lentil',
 'pound hardwood chips',
 'wood ear mushrooms',
 'corn bread crumbs',
 'skin on bone i

In [38]:
merged_df = ingr_list.apply(pd.Series)
merged_df = merged_df.rename(columns = lambda x: 'ingr_' + str(x))
merged_df.head(15)

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shortening,white sugar,salt,milk,egg,lemon,yeast,flour,fruit,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shortening,water,brown sugar,yeast,water,white sugar,salt,bread,whole wheat,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,water,cottage cheese,vegetable oil,egg,flour,white sugar,baking soda,salt,yeast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,margarine,white sugar,egg,banana,coffee,water,vanilla,flour,salt,baking powder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cornmeal,milk,egg,bell pepper,onion,garlic,salt,baking soda,white sugar,corn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
merged_df[~merged_df['ingr_64'].isnull()]

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
27640,fettucine,fresh marjoram,minced garlic,olive oil,garlic powder,large eggs,Alfredo sauce,vegetable oil,cajun seasoning,shredded romano cheese,...,boneless skinless chicken breasts,chicken cutlets,butter,multi-grain penne pasta,extra-virgin olive oil,cilantro leaves,green pepper,shredded mozzarella cheese,fresh parsley,spaghetti


In [137]:
(merged_df.isnull().mean() * 100).head(50)

ingr_0      0.000000
ingr_1      0.069278
ingr_2      0.667000
ingr_3      2.571332
ingr_4      6.173774
ingr_5     11.951216
ingr_6     19.618489
ingr_7     28.648762
ingr_8     38.076979
ingr_9     47.830640
ingr_10    57.033302
ingr_11    65.470686
ingr_12    72.807682
ingr_13    78.997567
ingr_14    84.048398
ingr_15    87.987562
ingr_16    91.190449
ingr_17    93.689281
ingr_18    95.559780
ingr_19    96.872835
ingr_20    97.902334
ingr_21    98.538723
ingr_22    98.980167
ingr_23    99.284667
ingr_24    99.487667
ingr_25    99.639111
ingr_26    99.735778
ingr_27    99.782500
ingr_28    99.837278
ingr_29    99.885611
ingr_30    99.914611
ingr_31    99.938778
ingr_32    99.948444
ingr_33    99.958111
ingr_34    99.964556
ingr_35    99.969389
ingr_36    99.977444
ingr_37    99.977444
ingr_38    99.982278
ingr_39    99.982278
ingr_40    99.987111
ingr_41    99.988722
ingr_42    99.988722
ingr_43    99.990333
ingr_44    99.990333
ingr_45    99.990333
ingr_46    99.990333
ingr_47    99

In [156]:
modelDF = merged_df.copy()

In [157]:
modelDF.columns[:15]

Index(['ingr_0', 'ingr_1', 'ingr_2', 'ingr_3', 'ingr_4', 'ingr_5', 'ingr_6',
       'ingr_7', 'ingr_8', 'ingr_9', 'ingr_10', 'ingr_11', 'ingr_12',
       'ingr_13', 'ingr_14'],
      dtype='object')

In [158]:
# drop columns ingr_15 - ingr_64

modelDF = modelDF[modelDF.columns[:15]]
modelDF

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,ingr_10,ingr_11,ingr_12,ingr_13,ingr_14
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,almond,orange juice,butter,almond,vanilla
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,raisin,egg,milk,vegetable oil,NaN
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62064,large egg yolks,fresh lemon juice,sugar,bourbon whiskey,bittersweet chocolate,large egg whites,salt,anjou pears,heavy whipping cream,NaN,NaN,NaN,NaN,NaN,NaN
62065,hot sauce,butter,sweet potatoes,adobo sauce,salt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62066,black pepper,salt,parmigiano reggiano cheese,ricotta,large egg yolks,dry bread crumbs,genoa salami,vegetable oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62067,cheddar cheese,cayenne,paprika,plum tomatoes,green bell pepper,water,Tabasco Pepper Sauce,scallions,canned low sodium chicken broth,milk,butter,red bell pepper,canned black beans,quickcooking grits,salt


In [159]:
modelDF.head()

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,ingr_10,ingr_11,ingr_12,ingr_13,ingr_14
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,almond,orange juice,butter,almond,vanilla
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,raisin,egg,milk,vegetable oil,NaN
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
derp_list = []
for i in range(len(modelDF)):
    row = modelDF.iloc[i]
    derp_list.append([item for item in row if item])

modelDF['ingredients'] = pd.Series(derp_list)
modelDF.head()

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,ingr_10,ingr_11,ingr_12,ingr_13,ingr_14,ingredients
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[yeast, water, white sugar, salt, egg, butter,..."
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,almond,orange juice,butter,almond,vanilla,"[flour, salt, baking powder, poppy, butter, ve..."
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,NaN,NaN,NaN,NaN,NaN,"[flour, egg, white sugar, vegetable oil, apple..."
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,raisin,egg,milk,vegetable oil,NaN,"[flour, baking powder, baking soda, salt, cinn..."
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,NaN,NaN,NaN,NaN,NaN,"[oat, buttermilk, vegetable oil, egg, brown su..."


In [174]:
def parse_ingredients(dfcolumn):
    '''Parses a column of ingredients and return a list of unique values'''

    # If instance is one string with all ingredients, split by comma
    if type(dfcolumn[0]) == str:
        dfcolumn = dfcolumn.apply(lambda x: x.split(','))

    # Create set of unique values, the convert to list
    ingredients = set(x for l in dfcolumn for x in l)
    ingredient_list = list(ingredients)

    return ingredient_list

In [175]:
trimmed_unique_ingr = parse_ingredients(modelDF['ingredients'])
ingr_int = {ingr:i for i, ingr in enumerate(trimmed_unique_ingr)}
int_ingr = {i:ingr for i, ingr in enumerate(trimmed_unique_ingr)}

In [177]:
len(ingr_int)

7711

In [ ]:
model_ingr_list = list(set())